# Лабораторная работа №3

In [244]:
import numpy as np
import math as m
import matplotlib.pyplot as plt

Общий класс метода и утилитарные методы:

In [264]:
import base64
import urllib.parse
import io
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from IPython.display import HTML, display
from abc import ABC, abstractmethod


class Method(ABC):

    # кортеж - (1, 2)
    @abstractmethod
    def result(self) -> tuple:
        pass

    # image - показывать ли изображение в таблице
    @abstractmethod
    def info(self, image=False) -> dict:
        pass


# В качестве особого ключа будет 'title', который будет указывать название метода
titleKey = 'title'
minPointKey = '1. Точка минимума'
minimumKey = '2. Минимум'
iterCountKey = '3. Кол-во итераций'
execCountKey = '4. Кол-во вызовов целевой функции'
gradCountKey = '5. Кол-во вычислений градиента'
hesseKey = '6. Кол-во вычислений матриц Гессе'
imageKey = '7. График приближения'

# возвращает строки таблиц (у каждой строки на первой позиции заголовок,
# далее идут значения) и заголовки


def dictsToRows(dicts: list[dict], defaultValue=''):
    keys = set()
    for dict in dicts:
        for key in dict.keys():
            keys.add(key)

    keys.discard('title')
    keys = list(keys)
    keys.sort()
    # формируем таблицу
    titles = [dict[titleKey] for dict in dicts]

    table = []
    for key in keys:
        row = [str(key)]
        for dict in dicts:
            cellData = defaultValue
            if dict.get(key) != None:
                cellData = str(dict[key])

            row.append(cellData)

        table.append(row)

    return table, titles


def dictsToTable(dicts: list[dict]):
    keys = set()
    for dict in dicts:
        for key in dict.keys():
            keys.add(key)

    keys.discard('title')
    keys = list(keys)
    keys.sort()
    # формируем таблицу
    table = '<th></th>'
    for dict in dicts:
        table += '<th>' + dict[titleKey] + '</th>'

    table = '<tr>' + table + '</tr>'

    for key in keys:
        table += '<tr>'
        table += '<th>' + str(key) + '</th>'
        for dict in dicts:
            cellData = ''
            if dict.get(key) != None:
                cellData = str(dict[key])

            table += '<th>' + cellData + '</th>'

        table += '</tr>'

    display(HTML('<table style="border:20px black solid">' + table + '</table>'))


def printMethodsInfo(methods: list[Method]):
    dictsToTable([method.info() for method in methods])


def figureToHtml(fig: Figure):
    imgdata = io.BytesIO()
    fig.savefig(imgdata, format='png', dpi=500)
    imgdata.seek(0)
    data = urllib.parse.quote(base64.b64encode(imgdata.read()).decode())
    return '<img src="data:image/png;base64,%s"/>' % data


def pltToHtml():
    fig = plt.gcf()
    data = figureToHtml(fig)
    # fig.clear()
    return data


def fmtFloat(num: float, eps: float) -> str:
    """
    Возвращает сторку с числом num, округленное до точености eps
    Пример: fmtFloat(0.125, 0.01) -> 0.13
    """
    count = -round(m.log10(eps))
    return f"{num:.{count}f}"


def drawPoints(fig: Figure, fun, points: list):
    minx = min(points, key=lambda x: x[0])[0]
    maxx = max(points, key=lambda x: x[0])[0]
    miny = min(points, key=lambda x: x[1])[1]
    maxy = max(points, key=lambda x: x[1])[1]

    deltax = (maxx - minx) / 10
    deltay = (maxy - miny) / 10

    if deltax < 0.1:
        deltax = 5
    if deltay < 0.1:
        deltay = 5

    minx -= deltax
    maxx += deltax
    miny -= deltay
    maxy += deltay

    # X = np.linspace(minx, maxx, num=200)
    # Y = np.linspace(miny, maxy, num=200)
    # X, Y = np.meshgrid(X, Y)
    # Z = []
    # for i in range(200):
    #     ZZ = []
    #     for j in range(200):
    #         ZZ.append(fun([X[i, j], Y[i, j]]))
    #     Z.append(ZZ)

    # ax = fig.subplots()
    # ax.grid()
    # ax.contourf(X, Y, Z)
    # ax.plot([x[0] for x in points], [x[1]
    #         for x in points], marker='o', markersize=3, color='red')

    pointsCount = 400
    X = np.linspace(minx, maxx, num=pointsCount)
    Y = np.linspace(miny, maxy, num=pointsCount)
    X, Y = np.meshgrid(X, Y)
    Z = []
    for i in range(pointsCount):
        ZZ = []
        for j in range(pointsCount):
            ZZ.append(fun([X[i, j], Y[i, j]]))
        Z.append(ZZ)

    ax = fig.subplots()
    ax.grid()
    maxLevels = 100
    levels = list(set([fun(x) for x in points[:min(maxLevels, len(points))]]))
    levels.sort()

    # maxPoints = 1000
    # points = points[:min(maxPoints, len(points))]

    ax.contour(X, Y, Z, levels=levels)
    ax.plot([x[0] for x in points], [x[1]
            for x in points], marker='o', markersize=3, color='red')


def showImage(fun, points: list, equal: bool = False):
    # перед вызовом этой функции нужно написать %matplotlib widget
    fig = plt.figure()
    drawPoints(fig, fun, points)
    figData = figureToHtml(fig)
    if equal:
        fig.gca().set_aspect('equal')

    # fig.show()
    # plt.close(fig)


# сохраняет картинку
def savefig(fun, points: list, filename, equal: bool = False):
    # перед вызовом этой функции нужно написать %matplotlib widget
    fig = plt.figure()
    drawPoints(fig, fun, points)
    figData = figureToHtml(fig)
    if equal:
        fig.gca().set_aspect('equal')

    fig.savefig(filename + '.png', dpi=600)
    plt.close(fig)


Минимизируемые функции:

In [26]:
def f1(x): return 10 * x[0]**2 - 4*x[0]*x[1] + 7 * \
    x[1]**2 - 4*m.sqrt(5) * (5*x[0]-x[1]) - 16


def f1Grad(x): return [-20 * m.sqrt(5) + 20 * x[0] -
                       4*x[1], 4*m.sqrt(5) - 4*x[0] + 14*x[1]]


def f1Hesse(x): return [[-20, -4], [-4, 14]]


def rozenbrok(alpha: float):
    # return lambda x: alpha * (x[0]**2 - x[1])**2 + (x[0]-1)**2
    return lambda x: alpha * np.power(np.power(x[0], 2) - x[1], 2) + np.power(x[0]-1, 2)


def rozenbrokGrad(alpha: float):
    # return lambda x: [-2*(1 - x[0]) - 4*x[0]*(-x[0]**2 + x[1])*alpha, 2*(-x[0]**2 + x[1])*alpha]
    return lambda x: [-2*(1 - x[0]) - 4*x[0]*(-np.power(x[0], 2) + x[1])*alpha, 2*(-np.power(x[0], 2) + x[1])*alpha]


def rozenbrokHesse(alpha: float):
    return lambda x: [[2.0 + 12*alpha*np.power(x[0], 2), -4*alpha*x[0]], [-4*alpha*x[0], 2*alpha]]


alpha1 = 1
alpha2 = 10

f2 = rozenbrok(alpha1)
f2Grad = rozenbrokGrad(alpha1)
f2Hesse = rozenbrokHesse(alpha1)

f3 = rozenbrok(alpha2)
f3Grad = rozenbrokGrad(alpha2)
f3Hesse = rozenbrokHesse(alpha2)


Метод золотого сечения для одномерной оптимизации, вторым параметром возвращает кол-во вычислений целевой функции:

In [293]:
def methodGoldenRatio(fun, a: float, b: float, eps: float):
    # tau = (m.sqrt(5) + 1) / 2
    tau1 = 2.0 / (3.0 + m.sqrt(5.0))
    tau2 = 2.0 / (1.0 + m.sqrt(5.0))
    ak, bk = a, b
    lk = bk - ak
    # xk1 = bk - lk / tau
    # xk2 = ak + lk / tau
    xk1 = ak + tau1 * lk
    xk2 = ak + tau2 * lk
    # y1, y2 = fun(xk1), fun(xk2)
    y1 = fun(xk1)
    y2 = fun(xk2)
    
    execCount = 2

    while lk >= eps/1000.0:
    # while np.abs(bk - ak) >= eps*(np.abs(xk1) + np.abs(xk2)) and execCount <= 5002:
    # while np.abs(bk - ak) >= eps*(np.abs(xk1) + np.abs(xk2)):
    # while execCount <= 5002:
    # mEps = eps*np.abs(b-a)
    # while np.abs(bk-ak) > mEps and execCount <= 5002:
    # while (np.abs(bk-ak) > mEps or m.isnan(y1) or m.isnan(y2)) and execCount <= 5002:
        if y1 < y2 or m.isnan(y2):
        # if y1 < y2:
            bk = xk2
            xk2 = xk1
            lk = bk - ak
            xk1 = ak + tau1 * lk
            y2 = y1
            y1 = fun(xk1)

            # if np.isnan(y1):
                # y1 = np.Inf
        else:
            ak = xk1
            xk1 = xk2
            lk = bk - ak
            xk2 = ak + tau2 * lk
            y1 = y2
            y2 = fun(xk2)

            # if np.isnan(y2):
                # y2 = np.Inf
        # lk = bk - ak
        execCount += 1
    return (ak + bk) / 2, execCount


In [242]:
#
from scipy.optimize import golden
def methodGoldenRatio(fun, a: float, b: float, eps: float):
    x, _, execCount = golden(fun, brack = (a, b), tol = eps, full_output=True)
    return x, execCount

Метод сопряженных градиентов:

In [289]:
ConjGradMethod = 1
FletcherReevesMethod = 2
PolakRieberMethod = 3

class ConjGradMethod(Method):
    methodName = 'Метод сопряженных градиентов'

    def __init__(self, fun, grad, hesse, eps: float, startPoint):
        self.points = []
        self.iterCount = 0
        self.execCount = 0
        self.gradCount = 0
        self.hesseCount = 0
        self.eps = eps

        self.fun = fun

        def w(x):
            gr = grad(x)
            return [-gr[0], -gr[1]]

        # Можно менять
        kappaMax = 1

        xk = startPoint
        self.points.append(xk)
        wk = w(xk)
        self.gradCount += 1
        lk = np.linalg.norm(wk)
        maxVecLen = 200.0

        p = wk

        while lk >= eps:
            self.iterCount += 1

            v = p
            if np.linalg.norm(v) >= maxVecLen:
                v = np.array(v) / np.linalg.norm(v) * maxVecLen

            # def phi(kappa): return fun(
            #     [xk[0] + kappa * p[0], xk[1] + kappa * p[1]])
            def phi(kappa): return fun(
                [xk[0] + kappa * v[0], xk[1] + kappa * v[1]])
            kappa, addExec = methodGoldenRatio(phi, 0, kappaMax, eps)
            self.execCount += addExec

            # newXk = [xk[0] + p[0]*kappa, xk[1] + p[1]*kappa]
            newXk = [xk[0] + v[0]*kappa, xk[1] + v[1]*kappa]

            newWk = w(newXk)
            self.gradCount += 1

            h = hesse(xk)
            self.hesseCount += 1

            gamma = 0
            if self.iterCount % 2 != 0:
                gamma = - np.dot(np.dot(h, p), newWk) / \
                    np.dot(np.dot(h, p), p)

            lk = np.linalg.norm(wk)
            xk = newXk
            wk = newWk
            p = [wk[0] + gamma*p[0], wk[1] + gamma*p[1]]

            self.points.append(xk)

        self.minPoint = xk
        self.minimum = fun(xk)

    def result(self) -> tuple:
        return self.minPoint, self.minimum

    def info(self, image=False) -> dict:
        d = {
            titleKey: self.methodName,
            minPointKey: '(' + fmtFloat(self.minPoint[0], self.eps) + ', ' + fmtFloat(self.minPoint[1], self.eps) + ')',
            minimumKey: fmtFloat(self.minimum, self.eps),
            iterCountKey: self.iterCount,
            execCountKey: self.execCount,
            gradCountKey: self.gradCount,
            hesseKey: self.hesseCount,
            # imageKey: figData
        }

        if image:
            fig = plt.figure()
            drawPoints(fig, self.fun, self.points)
            figData = figureToHtml(fig)
            plt.close(fig)
            d[imageKey] = figData

        return d


Метод Флетчера-Ривса:

In [290]:
class FletcherReevesMethod(Method):
    methodName = 'Метод Флетчера-Ривса'

    def __init__(self, fun, grad, eps: float, startPoint):
        self.points = []
        self.iterCount = 0
        self.execCount = 0
        self.gradCount = 0
        self.hesseCount = 0
        self.eps = eps

        self.fun = fun

        def w(x):
            gr = grad(x)
            return [-gr[0], -gr[1]]

        kappaMax = 1

        xk = startPoint
        self.points.append(xk)
        wk = w(xk)
        self.gradCount += 1
        lk = np.linalg.norm(wk)
        maxVecLen = 200.0

        p = wk

        while lk >= eps:
            self.iterCount += 1

            v = p
            if np.linalg.norm(v) >= maxVecLen:
                v = np.array(v) / np.linalg.norm(v) * maxVecLen
            
            # def phi(kappa): return fun(
                # [xk[0] + kappa * p[0], xk[1] + kappa * p[1]])
            def phi(kappa): return fun(
                [xk[0] + kappa * v[0], xk[1] + kappa * v[1]])
            
            kappa, addExec = methodGoldenRatio(phi, 0, kappaMax, eps)
            self.execCount += addExec

            # newXk = [xk[0] + p[0]*kappa, xk[1] + p[1]*kappa]
            newXk = [xk[0] + v[0]*kappa, xk[1] + v[1]*kappa]

            newWk = w(newXk)
            self.gradCount += 1

            gamma = 0
            if self.iterCount % 2 != 0:
                gamma = np.dot(newWk, newWk) / \
                    np.dot(wk, wk)

            lk = np.linalg.norm(wk)
            xk = newXk
            wk = newWk
            p = [wk[0] + gamma*p[0], wk[1] + gamma*p[1]]

            self.points.append(xk)

        self.minPoint = xk
        self.minimum = fun(xk)

    def result(self) -> tuple:
        return self.minPoint, self.minimum

    def info(self, image=False) -> dict:
        d = {
            titleKey: self.methodName,
            minPointKey: '(' + fmtFloat(self.minPoint[0], self.eps) + ', ' + fmtFloat(self.minPoint[1], self.eps) + ')',
            minimumKey: fmtFloat(self.minimum, self.eps),
            iterCountKey: self.iterCount,
            execCountKey: self.execCount,
            gradCountKey: self.gradCount,
            hesseKey: self.hesseCount,
            # imageKey: figData
        }

        if image:
            fig = plt.figure()
            drawPoints(fig, self.fun, self.points)
            figData = figureToHtml(fig)
            plt.close(fig)
            d[imageKey] = figData

        return d


Метод Полака-Рибера:

In [291]:
class PolakRieberMethod(Method):
    methodName = 'Метод Полака-Рибера'

    def __init__(self, fun, grad, eps: float, startPoint):
        self.points = []
        self.iterCount = 0
        self.execCount = 0
        self.gradCount = 0
        self.hesseCount = 0
        self.eps = eps

        self.fun = fun

        def w(x):
            gr = grad(x)
            return [-gr[0], -gr[1]]

        kappaMax = 1

        xk = startPoint
        self.points.append(xk)
        wk = w(xk)
        self.gradCount += 1
        lk = np.linalg.norm(wk)
        maxVecLen = 200.0

        p = wk

        while lk >= eps:
            self.iterCount += 1

            v = p
            if np.linalg.norm(v) >= maxVecLen:
                v = np.array(v) / np.linalg.norm(v) * maxVecLen
            
            # def phi(kappa): return fun(
                # [xk[0] + kappa * p[0], xk[1] + kappa * p[1]])
            def phi(kappa): return fun(
                [xk[0] + kappa * v[0], xk[1] + kappa * v[1]])
        
            kappa, addExec = methodGoldenRatio(phi, 0, kappaMax, eps)
            self.execCount += addExec

            # newXk = [xk[0] + p[0]*kappa, xk[1] + p[1]*kappa]
            newXk = [xk[0] + v[0]*kappa, xk[1] + v[1]*kappa]

            newWk = w(newXk)
            self.gradCount += 1

            gamma = 0
            if self.iterCount % 2 != 0:
                gamma = np.dot(np.array(newWk) - np.array(wk), newWk) / \
                    np.dot(wk, wk)

            lk = np.linalg.norm(wk)
            xk = newXk
            wk = newWk
            p = [wk[0] + gamma*p[0], wk[1] + gamma*p[1]]

            self.points.append(xk)

        self.minPoint = xk
        self.minimum = fun(xk)

    def result(self) -> tuple:
        return self.minPoint, self.minimum

    def info(self, image=False) -> dict:
        d = {
            titleKey: self.methodName,
            minPointKey: '(' + fmtFloat(self.minPoint[0], self.eps) + ', ' + fmtFloat(self.minPoint[1], self.eps) + ')',
            minimumKey: fmtFloat(self.minimum, self.eps),
            iterCountKey: self.iterCount,
            execCountKey: self.execCount,
            gradCountKey: self.gradCount,
            hesseKey: self.hesseCount,
            # imageKey: figData
        }

        if image:
            fig = plt.figure()
            drawPoints(fig, self.fun, self.points)
            figData = figureToHtml(fig)
            plt.close(fig)
            d[imageKey] = figData

        return d


Тесты:

Одиночный тест:

In [ ]:
import warnings
# warnings.filterwarnings('ignore')
# warnings.filterwarnings('error')
warnings.filterwarnings('default')

eps = 0.01
startPoint = [6., 7.]
fun = ['Функция Розенброка с \u03b1 = 10', f3, f3Grad, f3Hesse]
res1 = ConjGradMethod(fun[1], fun[2], fun[3], eps, startPoint)
printMethodsInfo([res1])
showImage(fun, res1.points)

Тест с созданием TeX'овского файла:

In [276]:
# меняем заголовки в техе, чтобы они влезали в ширину
texTitles = {
    minPointKey: 'Точка минимума',
    minimumKey: 'Минимум',
    iterCountKey: 'Кол-во итераций',
    execCountKey: '\\makecell{Кол-во вызовов\\\\целевой функции}',
    gradCountKey: '\\makecell{Кол-во вычислений\\\\градиента}',
    hesseKey: '\\makecell{Кол-во вычислений\\\\матриц Гессе}',
}

algNames = ['\\makecell{Метод\\\\сопряженных\\\\градиентов}',
            '\\makecell{Метод\\\\Флетчера --- Ривса}',
            '\\makecell{Метод\\\\Полака --- Рибера}']

imgFolder = './imgs/'


def texTest(funs, epss, startPoints, filename='test.tex'):
    # TODO получать названия методов
    algCount = len(algNames)
    text = ''

    for fun in funs:
        print(fun[0])
        text += "\\subsection{{{name}}}\n\n".format(name=fun[0])

        text += """\\begin{{table}}[H]
        \\centering
        \\vspace*{{-1.5em}}
        \\caption{{Результаты работы алгоритмов\\\\для {caption}}}
        \\footnotesize
        \\begin{{tabular}}{{{colsFormat}}}
        \\hline
        & {methods} \\\\
        \\hline
""".format(
            caption=fun[4],
            colsFormat='|' + 'c|'*(algCount + 2),
            methods=' '.join(['&' + name for name in algNames])
        )

        images = []
        imgText = ''
        for eps in epss:
            print('\u03b5 = ', eps)
            tables = []
            titles = []
            for point in startPoints:
                # TODO вынести вызов методов
                res1 = ConjGradMethod(
                    fun[1], fun[2], fun[3], eps, point)
                res1Info = res1.info()
                res2 = FletcherReevesMethod(fun[1], fun[2], eps, point)
                res2Info = res2.info()
                res3 = PolakRieberMethod(fun[1], fun[2], eps, point)
                res3Info = res3.info()

                baseName = ', eps ' + str(eps) + ', start = (' + fmtFloat(
                    point[0], eps) + ', ' + fmtFloat(point[1], eps) + ')' + ', ' + fun[0]
                baseName = baseName.translate(''.maketrans({'$': None, '\\': None}))
                baseCaption = 'Поиск минимума ' + fun[4] + ' при $\\varepsilon = ' + str(
                    eps) + '$, начальной точке (' + str(point[0]) + ', ' + str(point[1]) + ') '

                img1Name = res1.methodName + baseName
                img2Name = res2.methodName + baseName
                img3Name = res3.methodName + baseName

                img1Caption = baseCaption + 'методом сопряженных градиентов'
                img2Caption = baseCaption + 'методом Флетчера --- Ривса'
                img3Caption = baseCaption + 'методом Полака --- Рибера'

                savefig(fun[1], res1.points, imgFolder + img1Name)
                savefig(fun[1], res2.points, imgFolder + img2Name)
                savefig(fun[1], res3.points, imgFolder + img3Name)

                images.append([img1Name, img1Caption])
                images.append([img2Name, img2Caption])
                images.append([img3Name, img3Caption])

                table, titles = dictsToRows([res1Info, res2Info, res3Info])
                tables.append(table)

            text += """\t\\multirow{{{height}}}{{*}}{{\\rotatebox[origin=c]{{90}}{{$\\varepsilon = {epsVal}$}}}}""".format(
                    height=sum([len(table) for table in tables]),
                    epsVal=eps,
            )
            for ind, table in enumerate(tables):
                if ind != 0:
                    text += "\\cline{{2-{width}}}".format(width=2+len(titles))

                text += """&\\textbf{{Начальная точка}} &\\multicolumn{{{width}}}{{c|}}{{\\textbf{{{point}}}}}\\\\\n\t\\cline{{2-{widthTable}}}\n""".format(
                    width=len(titles),
                    point='(' + fmtFloat(startPoints[ind][0], eps) +
                    ', ' + fmtFloat(startPoints[ind][1], eps) + ')',
                    widthTable=2+len(titles),
                )

                for row in table:
                    row[0] = texTitles[row[0]]
                    text += """\t{row} \\\\ \n\t\\cline{{2-{width}}}\n""".format(
                        row=' '.join(['&' + cell for cell in row]),
                        width=2+len(titles),
                    )
            text += "\t\\hline\n"
        text += "\n\\end{tabular}\n\\end{table}\n\n"
        for img in images:
            imgText += """
            \\begin{{figure}}[H]
	        \\centering
	        \\includegraphics[width=0.70\\textwidth]{{{path}}}%
	        \\caption{{{caption}}}
	        \\vspace*{{-1.2cm}}
            \\end{{figure}}
            """.format(path=img[0], caption=img[1])
        text += imgText

    print('Записываем в файл...')
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(text)

Квадратичная функция
ε =  0.01
ε =  1e-06
Функция Розенброка с $\alpha$ = 1
ε =  0.01
ε =  1e-06
Функция Розенброка с $\alpha$ = 10
ε =  0.01
ε =  1e-06
Записываем в файл...


In [277]:
%%time
epss = [0.01, 1e-6]
funs = [['Квадратичная функция', f1, f1Grad, f1Hesse, 'квадратичной функции'],
        ['Функция Розенброка с $\\alpha$ = 1', f2, f2Grad,
            f2Hesse, 'функции Розенброка с $\\alpha$ = 1'],
        ['Функция Розенброка с $\\alpha$ = 10', f3, f3Grad, f3Hesse, 'функции Розенброка с $\\alpha$ = 10']]
# funs = [['Квадратичная функция', f1, f1Grad, f1Hesse, 'квадратичной функции']]
startPoints = [[3., 20.], [-5., 6.]]

texTest(funs, epss, startPoints)

Квадратичная функция
ε =  0.01
ε =  1e-06
Функция Розенброка с $\alpha$ = 1
ε =  0.01
ε =  1e-06
Функция Розенброка с $\alpha$ = 10
ε =  0.01
ε =  1e-06
Записываем в файл...
Wall time: 1min 2s


In [49]:
import warnings
warnings.filterwarnings('default')

Большой тест:

In [294]:
# import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings('error')

epss = [0.01, 1e-6]

funs = [['Квадратичная функция', f1, f1Grad, f1Hesse], ['Функция Розенброка с \u03b1 = 1',
                                                        f2, f2Grad, f2Hesse], ['Функция Розенброка с \u03b1 = 10', f3, f3Grad, f3Hesse]]

# funs = [['Функция Розенброка с \u03b1 = 1', f2, f2Grad, f2Hesse], ['Функция Розенброка с \u03b1 = 10', f3, f3Grad, f3Hesse]]


startPoints = [
    [10., 10.], [-100., -100.], [0., 0.], [1000., 1000.],
    [5., 1.], [20., -200.0], [50.0, 2.], [12., -5.],
    [1., 1.], [6., 7.], [2., 2.], [-5., -5.]
]

# startPoints = [
#     [5., 1.], [20., -200.0], [50.0, 2.], [12., -5.],
#     [1., 1.], [6., 7.], [2., 2.], [-5., -5.]
# ]

for i, fun in enumerate(funs):
    for j, eps in enumerate(epss):
        ind = 2*(i*len(epss) + j)
        # ind = i*len(epss) + j
        startPoint = startPoints[ind]

        print(fun[0] + ', \u03b5 = ' + str(eps) + ', начальная точка = (' +
              str(startPoint[0]) + ', ' + str(startPoint[1]) + ')')

        res1 = ConjGradMethod(fun[1], fun[2], fun[3], eps, startPoint)
        res2 = FletcherReevesMethod(fun[1], fun[2], eps, startPoint)
        res3 = PolakRieberMethod(fun[1], fun[2], eps, startPoint)
        printMethodsInfo([res1, res2, res3])

        ind += 1
        # ind = i*len(epss) + j
        startPoint = startPoints[ind]

        print(fun[0] + ', \u03b5 = ' + str(eps) + ', начальная точка = (' +
              str(startPoint[0]) + ', ' + str(startPoint[1]) + ')')

        res1 = ConjGradMethod(fun[1], fun[2], fun[3], eps, startPoint)
        res2 = FletcherReevesMethod(fun[1], fun[2], eps, startPoint)
        res3 = PolakRieberMethod(fun[1], fun[2], eps, startPoint)
        printMethodsInfo([res1, res2, res3])


Квадратичная функция, ε = 0.01, начальная точка = (10.0, 10.0)


,Метод сопряженных градиентов,Метод Флетчера-Ривса,Метод Полака-Рибера
1. Точка минимума,"(2.24, -0.00)","(2.24, 0.00)","(2.24, 0.00)"
2. Минимум,-66.00,-66.00,-66.00
3. Кол-во итераций,10,3,3
4. Кол-во вызовов целевой функции,260,78,78
5. Кол-во вычислений градиента,11,4,4
6. Кол-во вычислений матриц Гессе,10,0,0


Квадратичная функция, ε = 0.01, начальная точка = (-100.0, -100.0)


,Метод сопряженных градиентов,Метод Флетчера-Ривса,Метод Полака-Рибера
1. Точка минимума,"(2.24, -0.00)","(2.24, 0.00)","(2.24, 0.00)"
2. Минимум,-66.00,-66.00,-66.00
3. Кол-во итераций,11,3,3
4. Кол-во вызовов целевой функции,286,78,78
5. Кол-во вычислений градиента,12,4,4
6. Кол-во вычислений матриц Гессе,11,0,0


Квадратичная функция, ε = 1e-06, начальная точка = (0.0, 0.0)


...

,Метод сопряженных градиентов,Метод Флетчера-Ривса,Метод Полака-Рибера
1. Точка минимума,"(1.000000, 0.999999)","(1.000000, 1.000000)","(1.000000, 0.999999)"
2. Минимум,0.000000,0.000000,0.000000
3. Кол-во итераций,48,17,17
4. Кол-во вызовов целевой функции,2208,782,782
5. Кол-во вычислений градиента,49,18,18
6. Кол-во вычислений матриц Гессе,48,0,0


Функция Розенброка с α = 10, ε = 0.01, начальная точка = (1.0, 1.0)


,Метод сопряженных градиентов,Метод Флетчера-Ривса,Метод Полака-Рибера
1. Точка минимума,"(1.00, 1.00)","(1.00, 1.00)","(1.00, 1.00)"
2. Минимум,0.00,0.00,0.00
3. Кол-во итераций,0,0,0
4. Кол-во вызовов целевой функции,0,0,0
5. Кол-во вычислений градиента,1,1,1
6. Кол-во вычислений матриц Гессе,0,0,0


Функция Розенброка с α = 10, ε = 0.01, начальная точка = (6.0, 7.0)


KeyboardInterrupt: 

В зависимости:
1. от параметров точности;
2. начальной точки;
3. выпуклости (переход от квадратичной функции к функции Розенброка);
4. овражности функции (параметра α в функции Розенброка)

1. Для квадратичной функции лучше метод с дроблением шага, для Розенброка - наискорейший спуск.
2. Чем больше точность, тем больше вызовов целевой функции.
3. Выпуклость приводит к увеличению кол-во вызовов целевой функции и её градиента.
4. Увеличение овражности приводит к увеличению кол-во вызовов целевой функции и её градиента.